In [3]:
import os
import random 
import shutil 

# nurodome kelią į katalogus
data_dir = 'tomatoes_apples'
pomidorai_dir = os.path.join(data_dir, 'tomatoes')
obuoliai_dir = os.path.join(data_dir, 'apples')

# aprašysim, kokia dalis tenka mokymui, testams ir validacijai
splits = (0.7, 0.15, 0.15)

def split_data(directory, splits):
    """ 
    Funkcija yra skirta suskaidyti pateiktam kataloge esančias nuotraukas į tris naujus katalogus, pagal pateiktus išskaidymo dydžius
    Parametrai:
    directory = nuoroda iki failo kurį norite skaidyti
    splits = tuple, su nurodytais kiekiais mokymui, testavimui, validacijai
    """
    images = os.listdir(directory) #gauname visas nuotraukas
    random.shuffle(images) #išmaišome nuotraukas, siekiant skirtingu paleidimu metu turėti skirtingus numerius
    # print(images)
    train_size = int(len(images) * splits[0]) # sužinoma kiekius, kiek nuotraukų reikės mokymams
    validation_size = int(len(images) * splits[1])
    test_size = int(len(images) * splits[2])
    # print(train_size)
    # print(validation_size)
    # print(test_size)
    
    # katalogų nuorodų sukūrimas
    train_dir = os.path.join(directory, 'train')
    validation_dir = os.path.join(directory, 'validation')
    test_dir = os.path.join(directory, 'test')

    # os.removedirs(train_dir)
    # os.removedirs(validation_dir)
    # os.removedirs(test_dir)

    # TODO: sunaikinti katalogus, prieš tai sunaikinant turinį juose
    
    # katalogų sukūrimas pagal pateiktas nuorodas
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(validation_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    for i, image in enumerate(images):
        if i < train_size:
            shutil.copy(os.path.join(directory, image), os.path.join(train_dir, image))
        elif i < train_size + validation_size:
            shutil.copy(os.path.join(directory, image), os.path.join(validation_dir, image))
        else:
            shutil.copy(os.path.join(directory, image), os.path.join(test_dir, image))

       
split_data(pomidorai_dir, splits)
split_data(obuoliai_dir, splits)


In [4]:
# pasiekti nuotraukas
# patikrinti ar nuotrauka validi
# suvienodinti nuotraukas 

import os
from PIL import Image

data_dir = 'tomatoes_apples'
pomidorai_dir = os.path.join(data_dir, 'tomatoes')
obuoliai_dir = os.path.join(data_dir, 'apples')

# skirta patikrinti ar negausime klaidos, kaip argumentą pateikiame kelią iki nuotraukos
def is_valid_image(file_path):
    try:
        with Image.open(file_path) as img:    
            img.verify() # patikrina ar nuotrauką galima atidaryti
        return True
    except (IOError, SyntaxError):
        return False
    
def get_valid_image_files(directory):
    """
    Skirta direktorijoje esančius failus, patikrinti ar jie yra validūs. 
    Rezultatas - failų pavadinimų sąrašas, su validžių failų pavadinimas
    """
    valid_files = []
    # naudojame _, nes neketiname naudoti katalogų train, test ir validation
    for root, _, files in os.walk(directory):
        if root != directory:
        # panaikinti pirma direktorija
        # file = files[1:]
            for file in files:
                file_path = os.path.join(root, file)
                if is_valid_image(file_path):
                    valid_files.append(file_path)
    return valid_files

valid_pomidorai_photos = get_valid_image_files(pomidorai_dir)
valid_obuoliai_photos = get_valid_image_files(obuoliai_dir)



In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd

# naudojame siekiant sumažinti pixelių vertes iš intervalo 0-255 į intervalą 0-1
datagen = ImageDataGenerator(rescale=1./255)
# kuriame data frame, nes tai yra būdas perteikti informaciją generatoriui
pomidorai_df = pd.DataFrame({'filename':valid_pomidorai_photos})
obuoliai_df = pd.DataFrame({'filename':valid_obuoliai_photos})

pomidorai_generator = datagen.flow_from_dataframe(
    dataframe = pomidorai_df, #nurodome, kur yra musu nuotrauku sarasas
    x_col='filename', #nurodome, kuris stulpelis yra failo kelias musu df
    target_size = (150,150), #nurodome nuotraukų dydžius apkerpa, bet nesuspaudžia
    batch_size = 20, #nurodome, kiek nuotraukų įdėsime kiekvienos iteracijos metu
    class_mode = None) #turime tik du galimus outputus, todėl class mode nustatome none

obuoliai_generator = datagen.flow_from_dataframe(
    dataframe = obuoliai_df, #nurodome, kur yra musu nuotrauku sarasas
    x_col='filename', #nurodome, kuris stulpelis yra failo kelias musu df
    target_size = (150,150), #nurodome nuotraukų dydžius
    batch_size = 20, #nurodome, kiek nuotraukų įdėsime kiekvienos iteracijos metu
    class_mode = None) #turime tik du galimus outputus, todėl class mode nustatome none

Found 173 validated image filenames.
Found 218 validated image filenames.


In [20]:
from tensorflow.keras.utils import Sequence
import numpy as np
# kuriame šią klase tam, kad turėtume galimybę training matricas pateikti gabalais(batches)
class CombinedGenerator(Sequence):
    def __init__(self, *generators):
        self.generators =generators
        self._num_batches = sum(len(gen) for gen in generators)
        self.current_generator = 0

    def __len__(self):
        return self._num_batches
    
    def __getitem__(self, idx):
        for gen in self.generators:
            if idx < len(gen):
                batch = gen[idx]
                # generuojame labels, atsižvelgdami į tai, kiek narių turime savo batche
                labels = np.array([0]*batch.shape[0] if gen == pomidorai_generator else np.array([1] * batch.shape[0]))
                return batch, labels
            idx -= len(gen)            
    
combined_generator = CombinedGenerator(pomidorai_generator, obuoliai_generator) 

In [21]:
from tensorflow.keras import datasets, layers, models

model = models.Sequential((
    layers.Conv2D(32, (3,3), activation = 'relu', input_shape=(150, 150, 3)), #nurodom filtrus ir jų dydžių dimensiją(3,3)
    layers.MaxPooling2D(2,2), #sumažinam dimensijas, išlaikant svarbiausias savybes
    layers.Conv2D(64, (3, 3), activation='relu'), #antras konvoliucinis sluoksnis
    layers.MaxPooling2D((2,2)),  #sumažinam dimensijas, išlaikant svarbiausias savybes
    layers.Conv2D(128, (3,3), activation='relu'), 
    layers.MaxPooling2D((2,2)),  #sumažinam dimensijas, išlaikant svarbiausias savybes
    layers.Flatten(), #plokštinam duomenis, paversdami iš 3D į 1D
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid') # išėjimo sluoksnis su vienu neuronu, gražins tikimybę nou 0 iki 1  

))

c:\Users\edita\Documents\KURSAI\240319DMO492DIE\mokymai\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [22]:
model.compile(
    optimizer = 'adam', 
    loss='binary_crossentropy', #taikome, nes musu klasifikuojami bus 0 arba 1
    metrics=['accuracy']
)

In [23]:
history = model.fit(
    combined_generator, # nurodome generatorių, iš kurio duomenis imsime dalimis
    steps_per_epoch = len(combined_generator), #pasiims batchu kiekį
    epochs = 5
)

Epoch 1/5


c:\Users\edita\Documents\KURSAI\240319DMO492DIE\mokymai\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


20/20 ━━━━━━━━━━━━━━━━━━━━ 20s 761ms/step - accuracy: 0.5321 - loss: 3.3769
Epoch 2/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/5


C:\Users\edita\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


20/20 ━━━━━━━━━━━━━━━━━━━━ 13s 633ms/step - accuracy: 0.5358 - loss: 0.6886
Epoch 4/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 364us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/5
20/20 ━━━━━━━━━━━━━━━━━━━━ 13s 611ms/step - accuracy: 0.5344 - loss: 0.6946


In [24]:
def preprocess_image(image_path):
    img = Image.open(image_path)
    img = img.resize((150, 150))
    img_array = np.array(img)
    img_array = img_array.astype('float32') / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

In [25]:
def predict_image(image_path):
    img_array = preprocess_image(image_path)
    prediction = model.predict(img_array)
    return "Obuolys" if prediction[0][0] > 0.5 else 'Pomidoras'

print(predict_image('obuoliai1.jpg'))
print('obuolys--------------')
print(predict_image('obuoliai2.jpg'))
print('obuolys--------------')
print(predict_image('obuoliai3.jpg'))
print('obuolys--------------')
print(predict_image('pomidorai1.jpg'))
print('pomidoras--------------')
print(predict_image('pomidorai2.jpg'))
print('pomidoras--------------')
print(predict_image('pomidorai3.jpg'))
print('pomidoras--------------')
print(predict_image('slyva1.jpg'))
print('slyva--------------')
print(predict_image('slyva2.jpg'))
print('slyva--------------')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step
Obuolys
obuolys--------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Obuolys
obuolys--------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Obuolys
obuolys--------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Obuolys
pomidoras--------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Obuolys
pomidoras--------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Obuolys
pomidoras--------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Obuolys
slyva--------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Obuolys
slyva--------------


In [27]:
def preprocess_image(image_path):
    img = Image.open(image_path)
    if image_path.endswith(".png"):
        im = Image.open(image_path)
        img = im.convert('RGB')
        # rgb_im.save('colors.jpg')

    img = img.resize((150,150))
    img_array = np.array(img)
    img_array = img_array.astype('float32') / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    # print(img_array)
    return img_array

def predict_image(image_path):
    img_array = preprocess_image(image_path)
    prediction = model.predict(img_array)
    return "pomidoras" if prediction[0][0] > 0.5 else 'obuolys'